In [ ]:
using SparseArrays
using LinearAlgebra
using Arpack
using Statistics
using Random
using DelimitedFiles
using NPZ
using ExpmV

"""
function von_neumann_entropy(ψ::Vector{<:Complex}, cut::Int, L::Int)
    dimA = 3^cut
    dimB = 3^(L - cut)
    ψ_matrix = reshape(ψ, (dimA, dimB))
    svals = svdvals(ψ_matrix)
    S = 0.0
    for s in svals
        if s > 1e-15
            p = abs2(s)
            S -= p * log(p)
        end
    end
    return S
end
"""
function random_product_state(L::Int)
    product_state = nothing

    for i in 1:L
        θ1 = rand() * π
        θ2 = rand() * π
        ϕ1 = rand() * 2π
        ϕ2 = rand() * 2π

        c1 = cos(θ1 / 2)
        c2 = exp(im * ϕ1) * sin(θ1 / 2) * sin(θ2 / 2)
        c3 = exp(im * ϕ2) * sin(θ1 / 2) * cos(θ2 / 2)

        temp_state = [c1, c2, c3]

        if i == 1
            product_state = temp_state
        else
            product_state = kron(product_state, temp_state)
        end
    end

    # Normalize the state
    product_state /= norm(product_state)

    return product_state
end

"""
function time_evolution(ψ::Vector{ComplexF64}, dt::Float64, L)
    ψ /= norm(ψ)
    H, _, _ = Hamiltonian(L)
    a = 2π * rand()
    # Apply exp(-im * H * dt) directly to ψ
    ψ_new = expmv(-im * a * dt, H, ψ)

    # Normalize the state
    ψ_new /= norm(ψ_new)
    
    return ψ_new
end


function I3(psi::Vector{ComplexF64})
    log3(x) = log(x) / log(3)

    L = Int(round(log3(length(psi))))
    qL = L ÷ 4

    SA   = von_neumann_entropy(psi, qL, L)
    SB   = von_neumann_entropy(psi, 2qL, L)
    SC   = von_neumann_entropy(psi, 3qL, L)
    SABC = von_neumann_entropy(psi, L, L) ### seems quite sus
    SAB  = von_neumann_entropy(psi, 2qL, L)
    SAC  = von_neumann_entropy(psi, 3qL, L)
    SBC  = von_neumann_entropy(psi, 3qL, L)
    
    return SA + SB + SC + SABC - SAB - SAC - SBC
end
"""

function Hamiltonian(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^L
    Ha = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Ha += ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    Hb = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hb += (spl[i] * szl[i] * sml[ip] * szl[ip] + sml[i] * szl[i] * spl[ip] * szl[ip] + adjoint(spl[i] * szl[i] * sml[ip] * szl[ip]) + adjoint(sml[i] * szl[i] * spl[ip] * szl[ip]))
    end

    Hc = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hc += (spl[i]^2 * spl[ip] * szl[ip] + sml[i]^2 * sml[ip] * szl[ip] + adjoint(spl[i]^2 * spl[ip] * szl[ip]) + adjoint(sml[i]^2 * sml[ip] * szl[ip]))
    end

    return Ha, Hb, Hc, sz2l, szl 
end

### This function is for Z measurements only

function Entropy_t_z(L::Int, T::Float64, dt::Float64, p::Float64, shot::Int)
    Random.seed!(shot)  # Set random seed
    s_t = random_product_state(L)
    #s_t = neel_spin1_complex(L)  # Use the Néel state as the initial state 
    S_list = Float64[]

    # Define Hamiltonian and local observables
    _, _, _, sz2l, szl = Hamiltonian(L)
    sm_list = szl
    steps = Int(floor(T / dt))

    for _ in 1:steps
        #push!(S_list, entropy_vn(s_t, L, 1:L÷2)) ## Half-chain entropy
        push!(S_list, tmi(s_t))

        # Time evolution
        s_t = time_evolution(s_t, dt, L, shot)
        s_t /= norm(s_t)

        # Measurements
        if p != 0
            for l in 1:L
                x = rand()
                if x < p
                    p_m_mone = 0.5 * real(s_t' * (sz2l[l] - sm_list[l]) * s_t)
                    p_m_one = 0.5 * real(s_t' * (sz2l[l] + sm_list[l]) * s_t)
                    x1 = rand()
                    if x1 < p_m_mone
                        s_t = 0.5 * ((sz2l[l]-sm_list[l]) * s_t) / sqrt(p_m_mone)
                    elseif p_m_mone < x1 < p_m_one + p_m_mone
                        s_t = 0.5 * ((sz2l[l]+sm_list[l]) * s_t) / sqrt(p_m_one)
                    else
                        s_t = (s_t - sz2l[l] * s_t) / sqrt(1 - p_m_mone - p_m_one)
                    end
                end
            end
        end
    end

    # Save result to disk
    filename = "L$(L),T$(T),dt$(dt),p$(p),dirZ,s$(shot).npy"
    npzwrite(filename, S_list)
    

    """
    folder = "/Users/uditvarma/Documents/s3/data"
    filename = joinpath(folder, "L$(L),T$(T),dt$(dt),p$(p),dirZ,s$(shot).npy")
    npzwrite(filename, S_list)
    """
    
    return S_list
end

"""
function Hamiltonian(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^L
    H = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        H += ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    return H, sz2l, szl 
end
"""

function tmi(ψ::Vector{ComplexF64})
    log3(x) = log(x) / log(3)

    L = Int(round(log3(length(ψ))))
    qL = L ÷ 4

    SA   = entropy_vn(ψ, L, 1:qL)
    SB   = entropy_vn(ψ, L, (qL+1):(2qL))
    SC   = entropy_vn(ψ, L, (2qL+1):(3qL))
    SABC = entropy_vn(ψ, L, 1:3qL)
    SAB  = entropy_vn(ψ, L, 1:(2qL))
    SAC  = entropy_vn(ψ, L, (1:(qL)) ∪ ((2qL+1):3qL))
    SBC  = entropy_vn(ψ, L, (2qL+1):L)

    return SA + SB + SC - SAB - SAC - SBC + SABC

end

function entropy_vn(ψ::Vector{<:Complex}, L::Int, subsystem::AbstractArray{Int})
    cut = length(subsystem)
    dimA = 3^cut
    dimB = 3^(L - cut)
    ψ_matrix = reshape(ψ, (dimA, dimB))
    svals = svdvals(ψ_matrix)
    S = 0.0
    for s in svals
        if s > 1e-15
            p = abs2(s)
            S -= p * log(p)
        end
    end
    return S
end



function time_evolution(ψ::Vector{ComplexF64}, dt::Float64, L, shot::Int)
    Random.seed!(shot)  # Set random seed for reproducibility
    ψ /= norm(ψ)
    Ha, Hb, Hc, _, _ = Hamiltonian(L)
    a = 2π * rand()
    b = 2π * rand()
    c = 2π * rand()
    # Apply exp(-im * H * dt) directly to ψ
    H = a * Ha + b * Hb + c * Hc
    #H = Ha ## To test with Pradip's TDVP code
    ψ_new = expmv(-im * dt, H, ψ)

    # Normalize the state
    ψ_new /= norm(ψ_new)
    
    return ψ_new
end


# Function to generate spin-1 Néel state vector of length N as complex numbers
function neel_spin1_complex(N::Int)
    # Construct the Néel pattern: Up, Dn, Up, Dn...
    neel_state = [isodd(j) ? "Up" : "Dn" for j in 1:N]
    
    # Start with first spin
    psi = spin1_vector(neel_state[1])
    
    # Build the full tensor product
    for j in 2:N
        psi = kron(psi, spin1_vector(neel_state[j]))
    end
    
    # Convert to complex vector
    psi_complex = ComplexF64.(psi)  # broadcast conversion
    
    return psi_complex
end



neel_spin1_complex (generic function with 1 method)

In [21]:
### Add Lth spin to the Hamiltonian with the coupling as zeros

function Hamiltonian_an(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    J_list = [i < L-1 ? 1.0 : 0.0 for i in 1:L-1]

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^(L)
    Ha = spzeros(ComplexF64, dim, dim)

    for i in 1:L-1
        ip = (i  + 1)  # Open boundary
        Ha += J_list[i] * ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    Hb = spzeros(ComplexF64, dim, dim)

    for i in 1:L-1
        ip = (i + 1) # Open boundary
        Hb += J_list[i] * (spl[i] * szl[i] * sml[ip] * szl[ip] + sml[i] * szl[i] * spl[ip] * szl[ip] + adjoint(spl[i] * szl[i] * sml[ip] * szl[ip]) + adjoint(sml[i] * szl[i] * spl[ip] * szl[ip]))
    end

    Hc = spzeros(ComplexF64, dim, dim)

    for i in 1:L-1
        ip = (i + 1)  # Open boundary
        Hc += J_list[i] * (spl[i]^2 * spl[ip] * szl[ip] + sml[i]^2 * sml[ip] * szl[ip] + adjoint(spl[i]^2 * spl[ip] * szl[ip]) + adjoint(sml[i]^2 * sml[ip] * szl[ip]))
    end

    return Ha, Hb, Hc, sz2l, szl 
end

Hamiltonian_an (generic function with 1 method)

In [51]:
function Entropy_z_an(L::Int, dt::Float64, p::Float64, shot::Int)
    ## Change s_t to the Bell state as required
    s_t = spin1_bell(L)
    T   = 4 * L ## PRX paper
    _, _, _, sz2l, szl = Hamiltonian(L)
    sm_list = szl
    Sanc_list = Float64[]

    steps = Int(floor(T / dt))

    for n in 1:steps
        push!(Sanc_list, entropy_vn(s_t, L, 1:L-1)) ## Entropy of the ancilla spin

        # Time evolution
        s_t = time_evolution(s_t, dt, L, shot)

        # Effective measurement probability:
        #   = 0.0  for n ≤ steps/2
        #   = p    for n > steps/2
        p_eff = (n <= steps ÷ 2 ? 0.0 : p)

        # Measurements
        if p_eff != 0
            for l in 1:L-1
                x = rand()
                if x < p_eff
                    p_m_mone = 0.5 * real(s_t' * (sz2l[l] - sm_list[l]) * s_t)
                    p_m_one  = 0.5 * real(s_t' * (sz2l[l] + sm_list[l]) * s_t)
                    x1 = rand()
                    if x1 < p_m_mone
                        s_t = 0.5 * ((sz2l[l]-sm_list[l]) * s_t) / sqrt(p_m_mone)
                    elseif x1 < (p_m_mone + p_m_one)
                        s_t = 0.5 * ((sz2l[l]+sm_list[l]) * s_t) / sqrt(p_m_one)
                    else
                        s_t = (s_t - sz2l[l] * s_t) / sqrt(1 - p_m_mone - p_m_one)
                    end
                end
            end
        end
    end

    # Save result to disk
    filename = "L$(L),T$(T),dt$(dt),p$(p),dirZ,s$(shot)_anc.npy"
    npzwrite(filename, Sanc_list)
    return Sanc_list
end


Entropy_z_an (generic function with 1 method)

In [52]:
Entropy_z_an(7, 1.0, 0.0, 1)

28-element Vector{Float64}:
 0.6931471805599454
 1.097289740263696
 1.0972091381114282
 1.0941855473198856
 1.0975267838886824
 1.097972536116584
 1.0951135914148493
 1.097869488992679
 1.0969754151513662
 1.09367277062537
 ⋮
 1.0961220312771032
 1.0974753165467996
 1.0978810720810368
 1.0981632870609084
 1.09674657658484
 1.0983124119277008
 1.0931491685689103
 1.097436636429826
 1.0954680281131406

In [ ]:
### This creates a Bell pair state |Up, Down, ...> \kron |Up>_anc + |Down, Up, ...> \kron |Down>_anc

function spin1_bell(L::Int)
    dim = 3^L
    psi = zeros(ComplexF64, dim)

    # Helper: convert spin pattern to basis index
    # mapping: -1 -> 0, 0 -> 1, +1 -> 2
    function pattern_to_index(pattern)
        idx = 1
        for i in 1:L
            idx += (pattern[i]+1)*3^(L-i)
        end
        return idx
    end

    # Build the two patterns
    pattern1 = [i%2==1 ? 1 : -1 for i in 1:L-1]  # alternating pattern except last
    push!(pattern1, 1)  # enforce last spin = up

    pattern2 = [i%2==1 ? -1 : 1 for i in 1:L-1] # alternating pattern except last
    push!(pattern2, -1) # enforce last spin = down

    # Get computational basis indices
    idx1 = pattern_to_index(pattern1)
    idx2 = pattern_to_index(pattern2)

    # Set amplitudes
    psi[idx1] = 1/sqrt(2)
    psi[idx2] = 1/sqrt(2)

    return psi
end


spin1_bell (generic function with 1 method)

In [44]:
spin1_bell(3)

27-element Vector{ComplexF64}:
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865475 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                    ⋮
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865475 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im

In [75]:
include("s3_anc.jl")

neel_spin1_complex (generic function with 1 method)

In [76]:
Entropy_z_an(5, 1.0, 0.0, 900)

20-element Vector{Float64}:
 0.6931471805599454
 0.9467040629359256
 1.082006026494754
 1.090581916929082
 1.0768412233221376
 1.0861366164921837
 1.0931440309520037
 1.0551087347742338
 0.8999623349199521
 0.9081461709755846
 1.068878396120361
 1.0978068704091397
 1.0924723543651802
 1.087486926110826
 1.0957193029073138
 1.0898385067462422
 1.0419431301005928
 1.0060870667779211
 1.065176176545064
 1.0973792075302087

Running purification with initial state:
\begin{equation*}
    |\psi(t=0)\rangle = |\uparrow \downarrow \dots \rangle |\Uparrow\rangle + |\downarrow \uparrow \dots\rangle |\Downarrow\rangle
\end{equation*} 

In [69]:
include("s3_anc.jl")

for L in [5, 6, 7, 8, 9]
    for dt in [1.0]
        for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            for shot in 1:500
                Entropy_z_an(L, dt, p, shot)
            end
        end
    end
end

In [77]:
log(3)

1.0986122886681098

In [78]:
function spin1_ghz(L::Int)
    dim = 3^L
    psi = zeros(ComplexF64, dim)

    # Helper: convert spin pattern to basis index
    # mapping: -1 -> 0, 0 -> 1, +1 -> 2
    function pattern_to_index(pattern)
        idx = 1
        for i in 1:L
            idx += (pattern[i]+1)*3^(L-i)
        end
        return idx
    end

    # Pattern 1: Up,Down,Up,... and last spin = +1 (Up)
    pattern1 = [(i % 2 == 1 ? 1 : -1) for i in 1:L-1]
    push!(pattern1, 1)

    # Pattern 2: all zeros and last spin = 0
    pattern2 = fill(0, L-1)
    push!(pattern2, 0)

    # Pattern 3: Down,Up,Down,... and last spin = -1 (Down)
    pattern3 = [(i % 2 == 1 ? -1 : 1) for i in 1:L-1]
    push!(pattern3, -1)

    # Get computational basis indices
    idx1 = pattern_to_index(pattern1)
    idx2 = pattern_to_index(pattern2)
    idx3 = pattern_to_index(pattern3)

    # Set amplitudes (use 1 for an unnormalized sum, or divide by √3 to normalize)
    psi[idx1] = 1
    psi[idx2] = 1
    psi[idx3] = 1

    # Normalization:
    psi ./= sqrt(3)

    return psi
end


spin1_ghz (generic function with 1 method)

In [79]:
state = spin1_ghz(4)

81-element Vector{ComplexF64}:
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
     ⋮
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

In [81]:
entropy_vn(state, 4, 1:3)

1.0986122886681096

In [83]:
log(3)

1.0986122886681098

Running purification with initial state:
\begin{equation*}
    |\psi(t=0)\rangle = |\uparrow \downarrow \dots \rangle |\Uparrow\rangle + |00 \dots\rangle |0\rangle + |\downarrow \uparrow \dots\rangle |\Downarrow\rangle
\end{equation*} 

In [110]:
include("s3_anc.jl")

for L in [5, 6, 7, 8, 9]
    for dt in [1.0]
        for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            for shot in 1:500
                Entropy_z_an(L, dt, p, shot)
            end
        end
    end
end

The following code block ran for two days

In [ ]:
include("s3_anc.jl")

for L in [5, 6, 7, 8, 9]
    for dt in [1.0]
        for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            for shot in 1:500
                Entropy_z_an(L, dt, p, shot)
            end
        end
    end
end

### Checking the level spacing statistics of $S_3$ Hamiltonain

In [1]:
using SparseArrays
using LinearAlgebra
using Arpack
using Statistics
using Random
using DelimitedFiles
using NPZ
using ExpmV

In [4]:
function Hamiltonian(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^L
    Ha = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Ha += ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    Hb = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hb += (spl[i] * szl[i] * sml[ip] * szl[ip] + sml[i] * szl[i] * spl[ip] * szl[ip] + adjoint(spl[i] * szl[i] * sml[ip] * szl[ip]) + adjoint(sml[i] * szl[i] * spl[ip] * szl[ip]))
    end

    Hc = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hc += (spl[i]^2 * spl[ip] * szl[ip] + sml[i]^2 * sml[ip] * szl[ip] + adjoint(spl[i]^2 * spl[ip] * szl[ip]) + adjoint(sml[i]^2 * sml[ip] * szl[ip]))
    end

    return Ha, Hb, Hc, sz2l, szl 
end

Ha,_ , _, _, _ = Hamiltonian(8)

(sparse([7, 4375, 4378, 3, 19, 20, 21, 4381, 16, 4384  …  2178, 6546, 2181, 6541, 6542, 6543, 6559, 2184, 2187, 6555], [3, 3, 6, 7, 7, 8, 9, 9, 12, 12  …  6550, 6550, 6553, 6553, 6554, 6555, 6555, 6556, 6559, 6559], ComplexF64[1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im  …  1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im, 1.9999999999999982 + 0.0im], 6561, 6561), sparse([5, 2189, 3, 7, 2192, 5, 13, 14, 15, 2195  …  4367, 6547, 6548, 6549, 6557, 4370, 6555, 6559, 4373, 6557], [3, 3, 5, 5, 6, 7, 7, 8, 9, 9  …  6553, 6553, 6554, 6555, 6555, 6556, 6557, 6557, 

In [5]:
Ha

6561×6561 SparseMatrixCSC{ComplexF64, Int64} with 11664 stored entries:
⎡⠻⣦⡀⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⢬⠻⣦⡀⠀⠀⠀⠀⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⢻⣶⡀⢀⠀⠈⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⢈⡿⣯⣁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⢤⡀⠀⠁⠘⠻⢆⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣄⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠉⠀⠀⠀⠀⠉⣱⣾⡁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠈⠿⣧⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⠀⠀⠀⠀⠀⠀⠀⠙⠦⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠻⣦⡗⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠉⠻⣦⡀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡰⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠎⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠈⠻⣦⣀⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢼⠻⣦⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠲⣄⠀⠀⠀⠀⠀⠀⠀⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢻⣶⡀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢈⡿⢏⣀⠀⠀⠀⠀⣀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠙⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠱⣦⡄⢀⠀⠈⠓⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢉⣻⣾⡁⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡀⠀⠁⠈⠿⣧⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠀⠀⠀⠀⠈⠻⣦⡓⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠈⠻⣦⎦

In [4]:
using Pkg;
Pkg.add("StatsBase")
Pkg.add("Distributions")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
   Installed Rmath ─────────────────── v0.8.0
   Installed Rmath_jll ─────────────── v0.5.1+0
   Installed PDMats ────────────────── v0.11.35
   Installed StatsFuns ─────────────── v1.5.0
   Installed HypergeometricFunctions ─ v0.3.28
   Installed QuadGK ────────────────── v2.11.2
   Installed Distributions ─────────── v0.25.120
    Updating `~/.julia/environments/v1.11/Project.toml`
  [31c24e10] + Distributions v0.25.120
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [31c24e10] + Distributions v0.25.120
  [34004b35] + HypergeometricFunctions v0.3.28
  [90014a1f] + PDMats v0.11.35
  [1fd47b50] + QuadGK v2.11.2
  [79098fc4] + Rmath v0.8.0
  [4c63d2b9] + StatsFuns v1.5.0
  [f50d1b31] + Rmath_jll v0.5.1+0
  [4607b0f0] + SuiteSparse
Precompiling project...
    637.9 ms  ✓ Rmath_jll
   1448.4 ms  

In [5]:
# Level-spacing statistics in Julia
# - Accepts a Hamiltonian matrix H::AbstractMatrix or eigenvalues E::AbstractVector
# - Unfolds the spectrum with a Gaussian kernel estimate of N(E)
# - Plots histogram of spacings and overlays Poisson & Wigner surmise (GOE/GUE)
# - Prints the "r-statistic" (unfolding-free) for a quick chaos test

using LinearAlgebra
using StatsBase
using Distributions
using Plots

# ---------- helpers ----------
"""
    spectrum(H::AbstractMatrix) -> Vector{Float64}

Eigenvalues of a (Hermitian) Hamiltonian, sorted ascending.
If H is not exactly Hermitian, it’s symmetrized.
"""
function spectrum(H::AbstractMatrix)
    Hh = (H + H')/2
    vals = eigvals(Hh)
    sort!(real(vals))
    return vals
end

"""
    unfold(E; sigma=nothing, clip=0.1) -> Eunf

Unfold a sorted spectrum `E` using a Gaussian kernel estimate of the integrated density N(E).
Returns unfolded levels with mean spacing ~ 1. 
- `sigma`: kernel width; default is 1.06*std(E)*n^(-1/5) (Silverman-ish) but scaled for N(E).
- `clip`: drop this fraction from each edge before returning (edge effects).
"""
function unfold(E::AbstractVector{<:Real}; sigma=nothing, clip=0.1)
    @assert issorted(E) "E must be sorted. Use sort!(E) first."
    n = length(E)
    n >= 20 || @warn "Very short spectrum (n=$n): unfolding may be noisy."
    # bandwidth
    σ = sigma === nothing ? 1.06*std(E)*n^(-1/5) : sigma
    σ > 0 || error("sigma must be > 0")

    # Smooth integrated staircase: N_smooth(E) = sum Φ((E - Ei)/σ)
    # where Φ is the standard normal CDF.
    Φ = cdf.(Normal(), :)  # broadcastable
    Eunf = similar(E, Float64)
    for (k, Ek) in pairs(E)
        # This is O(n^2) but fine up to a few thousand levels.
        Eunf[k] = sum(Φ.((Ek .- E) ./ σ))
    end
    # Normalize so mean spacing ~ 1
    Eunf .*= (n-1) / (Eunf[end] - Eunf[1])
    # Optional: drop edges (they suffer most from boundary bias)
    m = round(Int, clip*n)
    if 2m < n
        return Eunf[(m+1):(n-m)]
    else
        return Eunf
    end
end

"""
    spacings(Eunf; tol=1e-10) -> s

Compute nearest-neighbor spacings from an unfolded sequence.
Filters out zero/near-zero spacings (due to degeneracies).
"""
function spacings(Eunf; tol=1e-10)
    s = diff(Eunf)
    s = s[s .> tol]
    # Normalize to mean 1 (extra safety)
    s ./= mean(s)
    return s
end

"""
    r_statistic(E; kclip=1) -> rmean

Unfolding-free ratio of consecutive spacings:
r_i = min(s_i, s_{i+1}) / max(s_i, s_{i+1}).
We drop `kclip` spacings from each edge to avoid boundary artifacts.
Reference means: Poisson≈0.3863, GOE≈0.5359, GUE≈0.6027.
"""
function r_statistic(E; kclip=1)
    @assert issorted(E) "E must be sorted."
    d = diff(E)
    d = d[d .> 0]
    if length(d) < 3
        return NaN
    end
    d = d[(kclip+1):(end-kclip)]  # light interior cut
    r = min.(@view(d[1:end-1]), @view(d[2:end])) ./ max.(@view(d[1:end-1]), @view(d[2:end]))
    return mean(r)
end

# Wigner surmise PDFs for overlay (unit mean spacings)
p_poisson(s) = @. exp(-s)
p_goe(s)     = @. (π/2)*s*exp(-π*s^2/4)
p_gue(s)     = @. (32/π^2)*s^2*exp(-4*s^2/π)

"""
    plot_level_spacing(E; nbins=50, show_gue=false, label="Your H")

Given a (sorted or unsorted) eigenvalue list E, unfolds, builds histogram of spacings
with unit mean, and overlays Poisson and GOE (and optionally GUE).
Returns the histogram data and the computed r-mean.
"""
function plot_level_spacing(E; nbins=50, show_gue=false, label="Your H")
    Es = sort(copy(E))
    rmean = r_statistic(Es)
    Eunf  = unfold(Es)
    s     = spacings(Eunf)

    # Histogram
    plt = histogram(s; bins=nbins, normalize=true, xlabel="s (unfolded spacing)",
                    ylabel="P(s)", label=label, title="Level Spacing Statistics")

    # Overlays
    sgrid = range(0, stop=3.0, length=400)
    plot!(sgrid, p_poisson.(sgrid), lw=2, label="Poisson (integrable)")
    plot!(sgrid, p_goe.(sgrid), lw=2, label="GOE Wigner–Dyson (time-reversal)")
    if show_gue
        plot!(sgrid, p_gue.(sgrid), lw=2, label="GUE Wigner–Dyson (no TRS)")
    end

    display(plt)
    println("r-statistic mean ≈ $(round(rmean, digits=4))  (Poisson≈0.3863, GOE≈0.5359, GUE≈0.6027)")
    return s, rmean
end

# ---------- convenience wrappers ----------
"""
    analyze_hamiltonian(H; kwargs...)

Compute eigenvalues and run the spacing analysis / plot.
"""
function analyze_hamiltonian(H; kwargs...)
    E = spectrum(H)
    return plot_level_spacing(E; kwargs...)
end

"""
    analyze_spectrum(E; kwargs...)

Run the spacing analysis / plot given precomputed eigenvalues.
"""
function analyze_spectrum(E; kwargs...)
    return plot_level_spacing(E; kwargs...)
end

# ---------- demo ----------
if abspath(PROGRAM_FILE) == @__FILE__
    # Example 1: GOE-like Hamiltonian (chaotic)
    N  = 400
    G  = randn(N, N)
    H_goe = (G + G')/sqrt(4N)  # symmetric, normalized
    println("GOE-like example:")
    analyze_hamiltonian(H_goe; nbins=60, show_gue=false, label="GOE-like H")

    # Example 2: Diagonal random Hamiltonian (integrable → Poisson)
    Ediag = sort!(rand(N))  # uncorrelated levels
    H_diag = Diagonal(Ediag)
    println("\nPoisson-like example:")
    analyze_hamiltonian(H_diag; nbins=60, show_gue=false, label="Diagonal (Poisson)")
end

In [6]:
# Level-spacing statistics in Julia
# - Works with dense and sparse Hamiltonians
# - Uses Arpack for sparse eigensolvers
# - Unfolds spectrum, plots level-spacing distribution
# - Prints r-statistic as chaos diagnostic

using LinearAlgebra
using StatsBase
using Distributions
using Plots
using Arpack   # for sparse eigensolvers

# ---------- helpers ----------
"""
    spectrum(H::AbstractMatrix; nev=nothing, which=:SM)

Compute sorted eigenvalues of Hamiltonian H.
- For dense H: computes all eigenvalues with `eigvals`.
- For sparse H: computes `nev` eigenvalues using ARPACK (`eigs`).
  `which` specifies part of spectrum (:SM=smallest magnitude, :LM=largest, :SR=smallest real, etc.)
"""
function spectrum(H::AbstractMatrix; nev=nothing, which=:SM)
    Hh = (H + H')/2   # Hermitize
    if issparse(Hh)
        if nev === nothing
            error("For sparse H, you must specify nev (number of eigenvalues).")
        end
        vals = eigs(Hh, nev=nev, which=which)[1]  # returns (vals, vecs, nconv, niter, nmult)
    else
        vals = eigvals(Hh)
    end
    return sort!(real(vals))
end

"""
    unfold(E; sigma=nothing, clip=0.1)

Unfold sorted spectrum `E` using Gaussian kernel estimate of integrated density N(E).
Returns unfolded levels with mean spacing ~ 1.
"""
function unfold(E::AbstractVector{<:Real}; sigma=nothing, clip=0.1)
    @assert issorted(E) "E must be sorted."
    n = length(E)
    σ = sigma === nothing ? 1.06*std(E)*n^(-1/5) : sigma
    Φ = cdf.(Normal(), :)
    Eunf = similar(E, Float64)
    for (k, Ek) in pairs(E)
        Eunf[k] = sum(Φ.((Ek .- E) ./ σ))
    end
    Eunf .*= (n-1) / (Eunf[end] - Eunf[1])
    m = round(Int, clip*n)
    if 2m < n
        return Eunf[(m+1):(n-m)]
    else
        return Eunf
    end
end

"""
    spacings(Eunf; tol=1e-10)

Nearest-neighbor spacings from unfolded sequence.
Filters out degeneracies.
"""
function spacings(Eunf; tol=1e-10)
    s = diff(Eunf)
    s = s[s .> tol]
    s ./= mean(s)  # normalize to mean 1
    return s
end

"""
    r_statistic(E; kclip=1)

Unfolding-free ratio of consecutive spacings:
r_i = min(s_i, s_{i+1})/max(s_i, s_{i+1}).
Reference: Poisson≈0.386, GOE≈0.536, GUE≈0.603.
"""
function r_statistic(E; kclip=1)
    @assert issorted(E) "E must be sorted."
    d = diff(E)
    d = d[d .> 0]
    if length(d) < 3
        return NaN
    end
    d = d[(kclip+1):(end-kclip)]
    r = min.(@view(d[1:end-1]), @view(d[2:end])) ./ max.(@view(d[1:end-1]), @view(d[2:end]))
    return mean(r)
end

# PDFs for overlay
p_poisson(s) = @. exp(-s)
p_goe(s)     = @. (π/2)*s*exp(-π*s^2/4)
p_gue(s)     = @. (32/π^2)*s^2*exp(-4*s^2/π)

"""
    plot_level_spacing(E; nbins=50, show_gue=false, label="")

Histogram of unfolded spacings + overlays.
Returns spacings and r-statistic.
"""
function plot_level_spacing(E; nbins=50, show_gue=false, label="spectrum")
    Es = sort(copy(E))
    rmean = r_statistic(Es)
    Eunf  = unfold(Es)
    s     = spacings(Eunf)

    plt = histogram(s; bins=nbins, normalize=true, xlabel="s", ylabel="P(s)",
                    label=label, title="Level Spacing Statistics")

    sgrid = range(0, stop=3.0, length=400)
    plot!(sgrid, p_poisson.(sgrid), lw=2, label="Poisson")
    plot!(sgrid, p_goe.(sgrid), lw=2, label="GOE")
    if show_gue
        plot!(sgrid, p_gue.(sgrid), lw=2, label="GUE")
    end

    display(plt)
    println("r-statistic mean ≈ $(round(rmean, digits=4))  (Poisson≈0.386, GOE≈0.536, GUE≈0.603)")
    return s, rmean
end

# ---------- wrappers ----------
function analyze_hamiltonian(H; nev=nothing, which=:SM, kwargs...)
    E = spectrum(H; nev=nev, which=which)
    return plot_level_spacing(E; kwargs...)
end

function analyze_spectrum(E; kwargs...)
    return plot_level_spacing(E; kwargs...)
end

# ---------- demo ----------
if abspath(PROGRAM_FILE) == @__FILE__
    # Example 1: GOE-like (dense)
    N  = 400
    G  = randn(N, N)
    H_goe = (G + G')/sqrt(4N)
    println("GOE-like example (dense):")
    analyze_hamiltonian(H_goe; nbins=60, label="GOE-like H")

    # Example 2: Poisson-like (diagonal random)
    Ediag = sort!(rand(N))
    H_diag = Diagonal(Ediag)
    println("\nPoisson-like example (dense diagonal):")
    analyze_hamiltonian(H_diag; nbins=60, label="Diagonal")

    # Example 3: Sparse random Hamiltonian
    using SparseArrays
    Nbig = 1000
    Hs = sprandn(Nbig, Nbig, 0.01)
    Hs = (Hs + Hs')/2  # Hermitian sparse
    println("\nSparse example (using 200 eigenvalues near smallest magnitude):")
    analyze_hamiltonian(Hs; nev=200, which=:SM, nbins=50, label="Sparse H")
end